In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('final_data.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [3]:
dummied = pd.get_dummies(df['position'])

In [4]:
df=df.drop(["player","name","team","position"], axis=1)
df= pd.concat([df, dummied], axis=1)
X = df.drop("current_value", axis=1)
y = df["current_value"]
df

,height,age,appearance,goals,assists,yellow_cards,second_yellow_cards,red_cards,goals_conceded,clean_sheets,...,defender_centre-back,defender_left-back,defender_right-back,goalkeeper,midfield,midfield-attackingmidfield,midfield-centralmidfield,midfield-defensivemidfield,midfield-leftmidfield,midfield-rightmidfield
0,189.000000,32.0,104,0.000000,0.000000,0.009585,0.0,0.000000,1.217252,0.335463,...,False,False,False,True,False,False,False,False,False,False
1,196.000000,30.0,15,0.000000,0.000000,0.069018,0.0,0.000000,1.242331,0.207055,...,False,False,False,True,False,False,False,False,False,False
2,188.000000,37.0,4,0.000000,0.000000,0.000000,0.0,0.000000,0.616438,0.924658,...,False,False,False,True,False,False,False,False,False,False
3,175.000000,25.0,82,0.028090,0.056180,0.224719,0.0,0.000000,0.000000,0.000000,...,True,False,False,False,False,False,False,False,False,False
4,191.000000,30.0,63,0.017889,0.017889,0.053667,0.0,0.000000,0.000000,0.000000,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10749,181.240353,20.0,16,0.175953,0.087977,0.263930,0.0,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
10750,190.000000,24.0,26,0.372671,0.186335,0.186335,0.0,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
10751,181.240353,19.0,20,0.375000,0.000000,0.187500,0.0,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
10752,181.240353,20.0,17,0.312139,0.104046,0.000000,0.0,0.104046,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)


c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [7]:
y_pred = xgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error: {rmse:.2f}")

Root Mean Squared Error: 2741529.14


In [8]:
import joblib
joblib.dump(xgb_model, 'xgboost_model.pkl')
# Assuming 'X_train' is your preprocessed training DataFrame
feature_names = X_train.columns.tolist()
joblib.dump(feature_names, 'feature_names.pkl')
feature_names


['height',
 'age',
 'appearance',
 'goals',
 'assists',
 'yellow_cards',
 'second_yellow_cards',
 'red_cards',
 'goals_conceded',
 'clean_sheets',
 'minutes_played',
 'days_injured',
 'games_injured',
 'award',
 'highest_value',
 'position_encoded',
 'winger',
 'attack',
 'attack-leftwinger',
 'attack-rightwinger',
 'attack-secondstriker',
 'attack_centre-forward',
 'defender',
 'defender_centre-back',
 'defender_left-back',
 'defender_right-back',
 'goalkeeper',
 'midfield',
 'midfield-attackingmidfield',
 'midfield-centralmidfield',
 'midfield-defensivemidfield',
 'midfield-leftmidfield',
 'midfield-rightmidfield']